In [1]:
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

%matplotlib inline

In [2]:
DATA_DIR_NAME = 'data'

In [3]:
labels = os.listdir(DATA_DIR_NAME)
labels.sort()
num_classes = len(labels)
labels[:10]

['Abra',
 'Aerodactyl',
 'Alakazam',
 'Alolan Sandslash',
 'Arbok',
 'Arcanine',
 'Articuno',
 'Beedrill',
 'Bellsprout',
 'Blastoise']

In [4]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [5]:
BATCH_SIZE = 64

generator = datagen.flow_from_directory(
    DATA_DIR_NAME,
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 6820 images belonging to 150 classes.


In [6]:
vgg16_base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg16_top_model = models.Sequential()
vgg16_top_model.add(layers.Flatten(input_shape=vgg16_base_model.output_shape[1:]))
vgg16_top_model.add(layers.Dense(256, activation='relu'))
vgg16_top_model.add(layers.Dense(num_classes, activation='softmax'))
vgg16_model = models.Model(inputs=vgg16_base_model.input, outputs=vgg16_top_model(vgg16_base_model.output))

In [7]:
for layer in vgg16_base_model.layers:
    layer.trainable = False

In [8]:
vgg16_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
vgg16_model.fit(
    generator,
    steps_per_epoch=generator.samples // BATCH_SIZE,
    epochs=12
)

In [ ]:
VGG16_MODEL_PATH = 'models/vgg16_model.keras'
vgg16_model.save(VGG16_MODEL_PATH)